In [20]:
import sys
from pathlib import Path
import warnings
warnings.filterwarnings("ignore", module="IPython")

#Find root directory
root_dir = Path().absolute()
if root_dir.parts[-2:] == ('notebooks', 'algae_bloom'):
    root_dir = Path(*root_dir.parts[:-2])
root_dir = str(root_dir)

print(f"Root dir: {root_dir}")

# Add the root directory to the `PYTHONPATH` 
if root_dir not in sys.path:
    sys.path.append(root_dir)
    print(f"Added the following directory to the PYTHONPATH: {root_dir}")

# Set the environment variables from the file <root_dir>/.env
from mlfs import config
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env") 

Root dir: /Users/kevinkokalari/Documents/Skalbar Maskininlärning och Djupinlärning/Laborationer/id2223-project
2025-12-18 14:50:47,017 WARNING: python-dotenv could not parse statement starting at line 12
HopsworksSettings initialized!


<span style="font-width:bold; font-size: 3rem; color:#333;">- Feature Backfill for Water Temperature Data</span>



## 🌐 Imports

In [21]:
import datetime
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
import datetime
from pathlib import Path
import json
import re
import os
import warnings
warnings.filterwarnings("ignore")

## Hopsworks connection


In [22]:
project = hopsworks.login(engine="python")

2025-12-18 14:50:47,040 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-12-18 14:50:47,044 INFO: Initializing external client
2025-12-18 14:50:47,044 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-12-18 14:50:48,523 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1272015


## 👉 Setting variables

In [23]:
today = datetime.date.today() 

backfill_file = f"{root_dir}/data/watertemp_midday_deduped.csv" #<-- ÄNDRA HÄR
util.check_file_path(backfill_file)

# Array of dictionaries for each bath location
bath_locations =[{'bath_location': 'Bergabadet', 'latitude': 59.057008, 'longitude': 17.440774}, 
                {'bath_location': 'Bränningestrand', 'latitude': 59.148617, 'longitude': 17.6674},
                {'bath_location': 'Eklundsnäsbadet', 'latitude': 59.16883, 'longitude': 17.59184},
                {'bath_location': 'Farstanäsbadet', 'latitude': 59.096884, 'longitude': 17.65387},
                {'bath_location': 'Mälarbadet', 'latitude': 59.222657, 'longitude': 17.611886},
                {'bath_location': 'Nya Malmsjöbadet', 'latitude': 59.234823, 'longitude': 17.536534},
                {'bath_location': 'Näsets udde(Glashyttan)', 'latitude': 59.158419, 'longitude': 17.66072},
                {'bath_location': 'Underåsbadet', 'latitude': 59.26482, 'longitude': 17.536534},
                {'bath_location': 'Åbynäsbadet', 'latitude': 59.018397, 'longitude': 17.619576},
                ]

#coordinates = settings.coordinates #<-- ÄNDRA HÄR

File successfully found at the path: /Users/kevinkokalari/Documents/Skalbar Maskininlärning och Djupinlärning/Laborationer/id2223-project/data/watertemp_midday_deduped.csv


## 🖼️ Create DataFrame for Water Temperature

The cell below will read up historical air quality data as a CSV file into a Pandas DataFrame

In [24]:
#Store all water temperature measures for all bath locations in dataframe
raw_data_df = pd.read_csv(backfill_file,  parse_dates=['formatted_time'], skipinitialspace=True)

raw_data_df

,temp_water,formatted_time,alias,latitude,longitude
0,2.9,2022-01-01 12:26:00,Bergabadet,59.057008,17.440774
1,2.8,2022-01-02 12:27:00,Bergabadet,59.057008,17.440774
2,2.8,2022-01-03 12:27:00,Bergabadet,59.057008,17.440774
3,2.7,2022-01-04 12:27:00,Bergabadet,59.057008,17.440774
4,2.8,2022-01-05 12:30:00,Bergabadet,59.057008,17.440774
...,...,...,...,...,...
6988,11.7,2025-05-16 14:58:00,Åbynäsbadet,59.018397,17.619576
6989,10.8,2025-05-17 14:58:00,Åbynäsbadet,59.018397,17.619576
6990,12.2,2025-05-19 14:58:00,Åbynäsbadet,59.018397,17.619576
6991,12.8,2025-05-20 14:58:00,Åbynäsbadet,59.018397,17.619576


## 🧹Clean data

In [25]:
#Select columns
water_temp_df = raw_data_df[['formatted_time', "alias", 'temp_water', 'longitude', "latitude"]] #<-- ÄNDRA

water_temp_df

,formatted_time,alias,temp_water,longitude,latitude
0,2022-01-01 12:26:00,Bergabadet,2.9,17.440774,59.057008
1,2022-01-02 12:27:00,Bergabadet,2.8,17.440774,59.057008
2,2022-01-03 12:27:00,Bergabadet,2.8,17.440774,59.057008
3,2022-01-04 12:27:00,Bergabadet,2.7,17.440774,59.057008
4,2022-01-05 12:30:00,Bergabadet,2.8,17.440774,59.057008
...,...,...,...,...,...
6988,2025-05-16 14:58:00,Åbynäsbadet,11.7,17.619576,59.018397
6989,2025-05-17 14:58:00,Åbynäsbadet,10.8,17.619576,59.018397
6990,2025-05-19 14:58:00,Åbynäsbadet,12.2,17.619576,59.018397
6991,2025-05-20 14:58:00,Åbynäsbadet,12.8,17.619576,59.018397


## 💧 Drop missing columns

In [26]:
water_temp_df.dropna(inplace=True)
water_temp_df

,formatted_time,alias,temp_water,longitude,latitude
0,2022-01-01 12:26:00,Bergabadet,2.9,17.440774,59.057008
1,2022-01-02 12:27:00,Bergabadet,2.8,17.440774,59.057008
2,2022-01-03 12:27:00,Bergabadet,2.8,17.440774,59.057008
3,2022-01-04 12:27:00,Bergabadet,2.7,17.440774,59.057008
4,2022-01-05 12:30:00,Bergabadet,2.8,17.440774,59.057008
...,...,...,...,...,...
6988,2025-05-16 14:58:00,Åbynäsbadet,11.7,17.619576,59.018397
6989,2025-05-17 14:58:00,Åbynäsbadet,10.8,17.619576,59.018397
6990,2025-05-19 14:58:00,Åbynäsbadet,12.2,17.619576,59.018397
6991,2025-05-20 14:58:00,Åbynäsbadet,12.8,17.619576,59.018397


## 🧺Add Values to Dataframe

In [27]:
#Add values to dataframe

#water_temp_df['country']= "sweden"#xx #<-- KOLLA OM DET BEHÖVS

water_temp_df

,formatted_time,alias,temp_water,longitude,latitude
0,2022-01-01 12:26:00,Bergabadet,2.9,17.440774,59.057008
1,2022-01-02 12:27:00,Bergabadet,2.8,17.440774,59.057008
2,2022-01-03 12:27:00,Bergabadet,2.8,17.440774,59.057008
3,2022-01-04 12:27:00,Bergabadet,2.7,17.440774,59.057008
4,2022-01-05 12:30:00,Bergabadet,2.8,17.440774,59.057008
...,...,...,...,...,...
6988,2025-05-16 14:58:00,Åbynäsbadet,11.7,17.619576,59.018397
6989,2025-05-17 14:58:00,Åbynäsbadet,10.8,17.619576,59.018397
6990,2025-05-19 14:58:00,Åbynäsbadet,12.2,17.619576,59.018397
6991,2025-05-20 14:58:00,Åbynäsbadet,12.8,17.619576,59.018397


In [28]:
water_temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6993 entries, 0 to 6992
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   formatted_time  6993 non-null   datetime64[ns]
 1   alias           6993 non-null   object        
 2   temp_water      6993 non-null   float64       
 3   longitude       6993 non-null   float64       
 4   latitude        6993 non-null   float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 273.3+ KB


## ⏮️ Calculate and Add Legged Values

In [29]:
#Sort to make sure lagged values are correct

water_temp_df = water_temp_df.sort_values(["alias", "formatted_time"]) # First sort by batch location, then date <-- KOLLA OM NAMN SKA ÄNDRAS

water_temp_df

,formatted_time,alias,temp_water,longitude,latitude
0,2022-01-01 12:26:00,Bergabadet,2.9,17.440774,59.057008
1,2022-01-02 12:27:00,Bergabadet,2.8,17.440774,59.057008
2,2022-01-03 12:27:00,Bergabadet,2.8,17.440774,59.057008
3,2022-01-04 12:27:00,Bergabadet,2.7,17.440774,59.057008
4,2022-01-05 12:30:00,Bergabadet,2.8,17.440774,59.057008
...,...,...,...,...,...
6988,2025-05-16 14:58:00,Åbynäsbadet,11.7,17.619576,59.018397
6989,2025-05-17 14:58:00,Åbynäsbadet,10.8,17.619576,59.018397
6990,2025-05-19 14:58:00,Åbynäsbadet,12.2,17.619576,59.018397
6991,2025-05-20 14:58:00,Åbynäsbadet,12.8,17.619576,59.018397


In [30]:
#Adding columns for lagged air quality

#Takes which sensor it is into account when running A
water_temp_df["lagged_wt_1_day"]  = water_temp_df.groupby("alias")["temp_water"].shift(1) #<-- KOLLA VARIABLESNAMN + ev felhantera
water_temp_df["lagged_wt_2_days"] = water_temp_df.groupby("alias")["temp_water"].shift(2)
water_temp_df["lagged_wt_3_days"] = water_temp_df.groupby("alias")["temp_water"].shift(3)

water_temp_df.dropna(inplace=True)
water_temp_df

,formatted_time,alias,temp_water,longitude,latitude,lagged_wt_1_day,lagged_wt_2_days,lagged_wt_3_days
3,2022-01-04 12:27:00,Bergabadet,2.7,17.440774,59.057008,2.8,2.8,2.9
4,2022-01-05 12:30:00,Bergabadet,2.8,17.440774,59.057008,2.7,2.8,2.8
5,2022-01-06 12:28:00,Bergabadet,2.8,17.440774,59.057008,2.8,2.7,2.8
6,2022-01-07 12:28:00,Bergabadet,2.8,17.440774,59.057008,2.8,2.8,2.7
7,2022-01-08 12:28:00,Bergabadet,2.8,17.440774,59.057008,2.8,2.8,2.8
...,...,...,...,...,...,...,...,...
6988,2025-05-16 14:58:00,Åbynäsbadet,11.7,17.619576,59.018397,10.9,11.0,13.4
6989,2025-05-17 14:58:00,Åbynäsbadet,10.8,17.619576,59.018397,11.7,10.9,11.0
6990,2025-05-19 14:58:00,Åbynäsbadet,12.2,17.619576,59.018397,10.8,11.7,10.9
6991,2025-05-20 14:58:00,Åbynäsbadet,12.8,17.619576,59.018397,12.2,10.8,11.7


In [31]:
#Double check
water_temp_df[water_temp_df["alias"] == "Mälarbadet"] #<-- FYLL I

,formatted_time,alias,temp_water,longitude,latitude,lagged_wt_1_day,lagged_wt_2_days,lagged_wt_3_days
3104,2022-01-04 12:09:00,Mälarbadet,2.1,17.611886,59.222657,2.2,2.2,2.2
3105,2022-01-05 12:10:00,Mälarbadet,2.0,17.611886,59.222657,2.1,2.2,2.2
3106,2022-01-06 12:10:00,Mälarbadet,1.6,17.611886,59.222657,2.0,2.1,2.2
3107,2022-01-07 12:10:00,Mälarbadet,1.4,17.611886,59.222657,1.6,2.0,2.1
3108,2022-01-08 12:10:00,Mälarbadet,1.5,17.611886,59.222657,1.4,1.6,2.0
...,...,...,...,...,...,...,...,...
3967,2025-12-12 13:36:00,Mälarbadet,6.3,17.611886,59.222657,6.4,6.5,6.4
3968,2025-12-13 13:36:00,Mälarbadet,6.1,17.611886,59.222657,6.3,6.4,6.5
3969,2025-12-14 13:36:00,Mälarbadet,6.1,17.611886,59.222657,6.1,6.3,6.4
3970,2025-12-15 13:36:00,Mälarbadet,6.3,17.611886,59.222657,6.1,6.1,6.3


---

<span style="font-width:bold; font-size: 3rem; color:#333;">- Feature Backfill for Weather Data</span>

## 🌦 Loading Weather Data from [Open Meteo](https://open-meteo.com/en/docs)

## <span style='color:#ff5f27'> 🌍 STEP 9: Download the Historical Weather Data </span>

https://open-meteo.com/en/docs/historical-weather-api#hourly=&daily=temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant

We will download the historical weather data for your `city` by first extracting the earliest date from your DataFrame containing the historical air quality measurements.

We will download all daily historical weather data measurements for your `city` from the earliest date in your air quality measurement DataFrame. It doesn't matter if there are missing days of air quality measurements. We can store all of the daily weather measurements, and when we build our training dataset, we will join up the air quality measurements for a given day to its weather features for that day. 

The weather features we will download are:

 * `temperature (average over the day)`
 * `precipitation (the total over the day)`
 * `wind speed (average over the day)`
 * `wind direction (the most dominant direction over the day)`


In [32]:
earliest_wt_date = pd.Series.min(water_temp_df['formatted_time'])
earliest_wt_date = earliest_wt_date.strftime('%Y-%m-%d')
earliest_wt_date

#Store all unique measure times
measure_times_df = water_temp_df[["formatted_time", "alias"]].copy()

weather_df = None

#Get weather for each location coordinates
for bath_dict in bath_locations: #<-- SE TILL ATT DETTA FUNGERAR MED VÅR LOGIK
        #Extract variables
        location = bath_dict["bath_location"]
        latitude = bath_dict["latitude"]
        longitude = bath_dict["longitude"]

        #Extract target times used for retrieving weather
        spec_measure_times_df = measure_times_df[measure_times_df["alias"] == location]

        #Get weather for each measurement for this location
        weather_df_temp = util.get_historical_weather(spec_measure_times_df, earliest_wt_date, str(today), latitude, longitude)
        
        #Update values in dataframe
        weather_df_temp["latitude"] = latitude
        weather_df_temp["longitude"] = longitude
        #weather_df_temp["batch_location"] = location

        #Merge with total weather datafram
        if weather_df is None:
            weather_df =  weather_df_temp
        else:
            weather_df = pd.concat([weather_df, weather_df_temp], ignore_index=True)


In [33]:


weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6966 entries, 0 to 6965
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   formatted_time      6966 non-null   datetime64[ns]
 1   alias               6966 non-null   object        
 2   rounded             6966 non-null   datetime64[ns]
 3   temperature_2m      6966 non-null   float32       
 4   precipitation       6966 non-null   float32       
 5   wind_speed_10m      6966 non-null   float32       
 6   wind_direction_10m  6966 non-null   float32       
 7   latitude            6966 non-null   float64       
 8   longitude           6966 non-null   float64       
dtypes: datetime64[ns](2), float32(4), float64(2), object(1)
memory usage: 381.1+ KB


In [34]:
weather_df.head()

,formatted_time,alias,rounded,temperature_2m,precipitation,wind_speed_10m,wind_direction_10m,latitude,longitude
0,2022-01-04 12:27:00,Bergabadet,2022-01-04 12:00:00,0.9525,0.0,15.188416,238.570480,59.057008,17.440774
1,2022-01-05 12:30:00,Bergabadet,2022-01-05 12:00:00,0.1025,0.0,7.289445,302.905243,59.057008,17.440774
2,2022-01-06 12:28:00,Bergabadet,2022-01-06 12:00:00,-4.8475,0.0,11.681987,303.690094,59.057008,17.440774
3,2022-01-07 12:28:00,Bergabadet,2022-01-07 12:00:00,-5.0475,0.0,4.104631,74.744827,59.057008,17.440774
4,2022-01-08 12:28:00,Bergabadet,2022-01-08 12:00:00,0.3025,0.0,5.351785,222.273621,59.057008,17.440774


## 📋 Define validation rules

In [43]:
#For water temperature data

import great_expectations as ge
wt_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="wt_expectation_suite"
)

wt_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column":"temp_water",
            "min_value":-5,
            "max_value":40,
            "strict_min":True
        }
    )
)

{"expectation_type": "expect_column_min_to_be_between", "kwargs": {"column": "temp_water", "min_value": -5, "max_value": 40, "strict_min": true}, "meta": {}}

In [ ]:
#For weather data

import great_expectations as ge
weather_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="weather_expectation_suite"
)

def expect_greater_than_zero(col):
    weather_expectation_suite.add_expectation(
        ge.core.ExpectationConfiguration(
            expectation_type="expect_column_min_to_be_between",
            kwargs={
                "column":col,
                "min_value":-0.1,
                "max_value":1000.0,
                "strict_min":True
            }
        )
    )
expect_greater_than_zero("precipitation")
expect_greater_than_zero("wind_speed_10m")

---

## 🤫 Connect to Hopsworks and save secrets

In [45]:
fs = project.get_feature_store() 

#### Save country, city, street names as a secret

These will be downloaded from Hopsworks later in the (1) daily feature pipeline and (2) the daily batch inference pipeline

In [46]:
#Store the name of the bath location and coordinates as secrets in Hopsworks

dict_array = []

for bath_dict in bath_locations:


    dict_array.append(bath_dict)
    print(dict_array)

#Convert to json
secret_value = json.dumps(dict_array)

#If secret already exists, we replace
secrets = hopsworks.get_secrets_api()

secret = secrets.get_secret("BATH_LOCATIONS_JSON")
if secret is not None:
    secret.delete()
    print("Replacing existing BATH_LOCATIONS_JSON")


secrets.create_secret("BATH_LOCATIONS_JSON", secret_value)

[{'bath_location': 'Bergabadet', 'latitude': 59.057008, 'longitude': 17.440774}]
[{'bath_location': 'Bergabadet', 'latitude': 59.057008, 'longitude': 17.440774}, {'bath_location': 'Bränningestrand', 'latitude': 59.148617, 'longitude': 17.6674}]
[{'bath_location': 'Bergabadet', 'latitude': 59.057008, 'longitude': 17.440774}, {'bath_location': 'Bränningestrand', 'latitude': 59.148617, 'longitude': 17.6674}, {'bath_location': 'Eklundsnäsbadet', 'latitude': 59.16883, 'longitude': 17.59184}]
[{'bath_location': 'Bergabadet', 'latitude': 59.057008, 'longitude': 17.440774}, {'bath_location': 'Bränningestrand', 'latitude': 59.148617, 'longitude': 17.6674}, {'bath_location': 'Eklundsnäsbadet', 'latitude': 59.16883, 'longitude': 17.59184}, {'bath_location': 'Farstanäsbadet', 'latitude': 59.096884, 'longitude': 17.65387}]
[{'bath_location': 'Bergabadet', 'latitude': 59.057008, 'longitude': 17.440774}, {'bath_location': 'Bränningestrand', 'latitude': 59.148617, 'longitude': 17.6674}, {'bath_locatio

Secret('BATH_LOCATIONS_JSON', 'PRIVATE')

## 🎨 Create feature groups

In [ ]:
current_version = 1

water_temperature_fg = fs.get_or_create_feature_group(
    name='water_temperature',
    description='Water temperature at batch locations in Södertälje each day',
    version=current_version,
    primary_key=['alias'],
    event_time="formatted_time",
    expectation_suite=wt_expectation_suite
)

In [48]:
#Insert water temperature dataframe into feature group
water_temperature_fg.insert(water_temp_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1272015/fs/1258614/fg/1869151
2025-12-18 14:55:33,799 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1272015/fs/1258614/fg/1869151


Uploading Dataframe: 100.00% |██████████| Rows 6966/6966 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: water_temperature_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1272015/jobs/named/water_temperature_1_offline_fg_materialization/executions


(Job('water_temperature_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "temp_water",
           "min_value": -5,
           "max_value": 40,
           "strict_min": true
         },
         "meta": {
           "expectationId": 797708
         }
       },
       "result": {
         "observed_value": -0.9,
         "element_count": 6966,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-12-18T01:55:33.000799Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successfu

In [49]:
#Descriptions of features

water_temperature_fg.update_feature_description("formatted_time", "Date and time of measurement of water temperature")
water_temperature_fg.update_feature_description("alias", "Name of the bath location where the water temperature was measured")
water_temperature_fg.update_feature_description("latitude", "Latitude of sensor measuring water temperature")
water_temperature_fg.update_feature_description("longitude", "Longitude of sensor measuring water temperature")
water_temperature_fg.update_feature_description("temp_water", "Water temperature measured")
water_temperature_fg.update_feature_description("lagged_wt_1_day", "Water temperature measured yesterday")
water_temperature_fg.update_feature_description("lagged_wt_2_days", "Water temperature measured two days ago")
water_temperature_fg.update_feature_description("lagged_wt_3_days", "Water temperature measured three days ago")

In [ ]:

#Create feature group for whether data

w_version = 1

weather_fg = fs.get_or_create_feature_group(
    name='weather',
    description='Weather characteristics of each day at point in time corresponding to measurement',
    version=w_version,
    primary_key=['alias'],
    event_time="formatted_time",
    expectation_suite=weather_expectation_suite
)  

#### Insert the DataFrame into the Feature Group

In [51]:
#Insert wather dataframe into feature group
weather_fg.insert(weather_df, wait=True)

FeatureStoreException: Features are not compatible with Feature Group schema: 
 - date (type: 'timestamp') is missing from input dataframe.
 - temperature_2m_mean (type: 'float') is missing from input dataframe.
 - precipitation_sum (type: 'float') is missing from input dataframe.
 - wind_speed_10m_max (type: 'float') is missing from input dataframe.
 - wind_direction_10m_dominant (type: 'float') is missing from input dataframe.
 - city (type: 'string') is missing from input dataframe.
 - formatted_time (type: 'timestamp') does not exist in feature group.
 - alias (type: 'string') does not exist in feature group.
 - rounded (type: 'timestamp') does not exist in feature group.
 - temperature_2m (type: 'float') does not exist in feature group.
 - precipitation (type: 'float') does not exist in feature group.
 - wind_speed_10m (type: 'float') does not exist in feature group.
 - wind_direction_10m (type: 'float') does not exist in feature group.
 - latitude (type: 'double') does not exist 

In [ ]:
#Description of features in feature group

weather_fg.update_feature_description("formatted_time", "Date and time of measurement of weather")
weather_fg.update_feature_description("alias", "Name of bath_location where weather is measured/forecast for")
weather_fg.update_feature_description("temperature_2m", "Temperature in Celsius")
weather_fg.update_feature_description("precipitation", "Precipitation (rain/snow) in mm")
weather_fg.update_feature_description("wind_speed_10m", "Wind speed at 10m abouve ground")
weather_fg.update_feature_description("wind_direction_10m", "Dominant Wind direction over the dayd")

---